In [1]:
import pandas as pd
from tqdm import tqdm
from transformers import BertForMaskedLM, BertTokenizer, DataCollatorForLanguageModeling
from datasets import Dataset
import torch
import torch.nn as nn
from torch.utils.data import Dataset as TorchDataset, DataLoader

import sys
sys.path.append('..')
from src.data.preprocess import put_mask_with_classifier, get_toxicity
from src.models.predict import detoxificate_text_with_classifier
from src.models.train import train, train_classifier, evaluate_classifier
from src.models.metrics import semantic_similarity, style_accuracy, fluency, j_metric

import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 1337
torch.manual_seed(RANDOM_SEED)

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
model_name = "../models/bert_maskedlm"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)

In [3]:
class ToxicWordsDataset(TorchDataset):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

        positive_words = open('../data/external/positive_words.txt').read().split('\n')
        negative_words = open('../data/external/negative_words.txt').read().split('\n')
        toxic_words = open('../data/external/toxic_words.txt').read().split('\n')
        toxic_words.extend(negative_words)

        toxic_words = [w for w in toxic_words if w.isalnum() and len(w) > 1]
        positive_words = [w for w in positive_words if w.isalnum() and len(w) > 1]

        self.texts = []
        self.labels = []

        for w in tqdm(toxic_words):
            word = self.tokenizer(w, add_special_tokens=False, max_length=1, truncation=True).input_ids
            self.texts.append(word[0])
            self.labels.append(1)

        for w in tqdm(positive_words):
            word = self.tokenizer(w, add_special_tokens=False, max_length=1, truncation=True).input_ids
            self.texts.append(word[0])
            self.labels.append(0)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]
    
dataset = ToxicWordsDataset(tokenizer)

100%|██████████| 1904/1904 [00:00<00:00, 6141.47it/s]


In [4]:
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [5]:
class ToxicWordsClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(embedding_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x
    
toxicity_classifier = ToxicWordsClassifier(dataset.tokenizer.vocab_size, 512, 0.2)

In [6]:
EPOCHS = 10

optimizer = torch.optim.Adam(toxicity_classifier.parameters(), lr=1e-5)
criterion = nn.BCELoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(EPOCHS):
    train_classifier(epoch, toxicity_classifier, optimizer, criterion, train_dataloader, device)
    evaluate_classifier(epoch, toxicity_classifier, criterion, val_dataloader, device)

torch.save(toxicity_classifier.state_dict(), '../models/toxicity_classifier.pth')

	Epoch: 9, Loss: 0.5342, Acc: 0.7826: 100%|██████████| 22/22 [00:00<00:00, 333.31it/s]


In [7]:
toxic_word = 'buttcheeks'
non_toxic_word = 'university'
print(f'Word: {toxic_word}, Toxicity: {get_toxicity(toxic_word, tokenizer, toxicity_classifier)}')
print(f'Word: {non_toxic_word}, Toxicity: {get_toxicity(non_toxic_word, tokenizer, toxicity_classifier)}')

Word: buttcheeks, Toxicity: 0.9843496680259705
Word: university, Toxicity: 0.15710808336734772


In [8]:
df = pd.read_csv('../data/interim/filtered.csv')
toxic_sentences = df['reference'].tolist()
non_toxic_sentences = df['translation'].tolist()
toxic_words = open('../data/interim/toxic_words.txt').read().split('\n')

data = []
labels = []

for i in tqdm(range(len(toxic_sentences))):
    toxic_sentences[i] = put_mask_with_classifier(toxic_sentences[i], tokenizer, toxicity_classifier)
    if '[MASK]' in toxic_sentences[i]:
        data.append(toxic_sentences[i])
        labels.append(non_toxic_sentences[i])

dataset = Dataset.from_dict({"text": data, "labels": labels})

 11%|█         | 11211/101535 [00:39<04:45, 316.13it/s]

list index out of range



100%|██████████| 101535/101535 [06:36<00:00, 255.96it/s]


In [9]:
MAX_LEN = 128

def group_texts(examples):
    inputs = [ex for ex in examples['text']]
    target = [ex for ex in examples['labels']]

    batch = tokenizer(inputs, padding='max_length', max_length=MAX_LEN, truncation=True, return_tensors='pt')
    batch["labels"] = tokenizer(target, padding='max_length', max_length=MAX_LEN, truncation=True, return_tensors='pt').input_ids

    return batch

dataset = dataset.map(group_texts, batched=True)

Map:   0%|          | 0/100422 [00:00<?, ? examples/s]

In [10]:
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset = dataset.select(range(train_size))
val_dataset = dataset.select(range(train_size, train_size + val_size))

In [12]:
train('maskedlm', 
      model, 
      tokenizer, 
      train_dataset, 
      val_dataset, 
      data_collator,
      batch_size=16, 
      epochs=1,
      seed=RANDOM_SEED
)

  0%|          | 0/5649 [00:00<?, ?it/s]

{'loss': 3.4868, 'learning_rate': 1.8229775181448045e-05, 'epoch': 0.09}


  0%|          | 0/628 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.63 GiB (GPU 0; 6.00 GiB total capacity; 2.99 GiB already allocated; 0 bytes free; 4.38 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [13]:
import random

random.seed(RANDOM_SEED)

best_model = BertForMaskedLM.from_pretrained("../models/bert_maskedlm")
tokenizer = BertTokenizer.from_pretrained("../models/bert_maskedlm")

random_toxic_sentences = random.sample(df['reference'].tolist(), 3)

for sentence in random_toxic_sentences:
    print(f'Original: {sentence}')
    print(f'Masked: {put_mask_with_classifier(sentence, tokenizer, toxicity_classifier)}')
    print(f'Detoxified: {detoxificate_text_with_classifier(sentence, tokenizer, best_model, toxicity_classifier)}')
    print()

Original: Suddenly, to the delight and outrage of the congregation, a raucous saxophone broke the solemnity, and a jazz rendering of "Fools Rush In" was blaring over the loudspeakers.
Masked: [MASK] [MASK] the delight and [MASK] [MASK] the congregation, [MASK] [MASK] [MASK] [MASK] the [MASK] and [MASK] jazz rendering [MASK] "fools [MASK] [MASK] [MASK] [MASK] [MASK] the [MASK]
Detoxified: and, the delight and joy from the congregation, the the ed, the blues and the jazz renderings " fools, " " " and the world

Original: This place is such a dump.
Masked: this place is such [MASK] [MASK]
Detoxified: this place is such fun!

Original: Doesn't mean a damn thing!
Masked: doesn't [MASK] [MASK] [MASK] thing!
Detoxified: doesn't know the ing thing!

